In [ ]:
import scipy.io
import math
import os
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import re
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.python.client import device_lib
import random
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import natsort
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from scipy import signal
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp
import neptune
from decouple import config
from keras.callbacks import Callback
from keras.layers import Conv2D, Add


In [ ]:

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(device_lib.list_local_devices())

strategy = tf.distribute.MirroredStrategy(devices=["GPU:1"])
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

import neptune.new as neptune

run = neptune.init(
    project="abdalraheem.ijjeh/Signal-prediction",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhZGJjNTcyNC0yN2ViLTQ5YzctOGFkZC1jODNlZmU1Y2Q4ZDcifQ==",
)  # your credentials


In [ ]:


# Loading dataset

os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project')
mat = scipy.io.loadmat('dataset1_1555samples_S1_ANN 5-2x1000-400_valFold 1 of 10.mat')

x_train = mat['inputs_training']
y_train = mat['S1_training']

print(x_train.shape)

print(y_train.shape)

x_val = x_train[-155:]
y_val = y_train[-155:]

x_train = x_train[:-155]
y_train = y_train[:-155]

x_test = mat['inputs_testing']
print(x_test.shape)
y_test = mat['S1_testing']
print(y_test.shape)

batches = 8

train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batches)
val_set = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batches)
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batches)


In [ ]:

# ANN model


inputs = tf.keras.Input(shape=(5,))

first_hidden_l = tf.keras.layers.Dense(5)(inputs)
# BN = tf.keras.layers.BatchNormalization()(first_hidden_l)
# dropout = tf.keras.layers.Dropout(0.)(first_hidden_l)

second_hidden_l = tf.keras.layers.Dense(128,
                                        kernel_regularizer=tf.keras.regularizers.L1(0.01),
                                        activity_regularizer=tf.keras.regularizers.L2(0.01))(first_hidden_l)
# BN = tf.keras.layers.BatchNormalization()(second_hidden_l)
dropout = tf.keras.layers.Dropout(0.4)(second_hidden_l)

third_hidden_l = tf.keras.layers.Dense(256,
                                       kernel_regularizer=tf.keras.regularizers.L1(0.01),
                                       activity_regularizer=tf.keras.regularizers.L2(0.01))(dropout)
# BN = tf.keras.layers.BatchNormalization()(third_hidden_l)
dropout = tf.keras.layers.Dropout(0.5)(third_hidden_l)

fourth_hidden_l = tf.keras.layers.Dense(512,
                                        kernel_regularizer=tf.keras.regularizers.L1(0.01),
                                        activity_regularizer=tf.keras.regularizers.L2(0.01))(dropout)
# BN = tf.keras.layers.BatchNormalization()(fourth_hidden_l)
dropout = tf.keras.layers.Dropout(0.5)(fourth_hidden_l)

fifth_hidden_l = tf.keras.layers.Dense(400)(dropout)
# BN = tf.keras.layers.BatchNormalization()(fifth_hidden_l)
# dropout = tf.keras.layers.Dropout(0.4)(fifth_hidden_l)
# 
# sixth_hidden_l = tf.keras.layers.Dense(256)(dropout)
# # BN = tf.keras.layers.BatchNormalization()(sixth_hidden_l)
# dropout = tf.keras.layers.Dropout(0.4)(sixth_hidden_l)
# 
# seventh_hidden_l = tf.keras.layers.Dense(512)(dropout)
# # BN = tf.keras.layers.BatchNormalization()(seventh_hidden_l)
# dropout = tf.keras.layers.Dropout(0.4)(seventh_hidden_l)

output_l = tf.keras.layers.Dense(400)(fifth_hidden_l)
model = Model(inputs, output_l, name="ANN_model")
model.summary()


In [ ]:

# CNN model


inputs_1 = tf.keras.Input(shape=(400,1))

x = tf.keras.layers.LSTM(64, return_sequences=True)(inputs_1)
x = tf.keras.layers.LSTM(128, padding='same', return_sequences=True)(x)
x = tf.keras.layers.LSTM(256, padding='same', return_sequences=False)(x)


rnn_model = tf.keras.models.Model(inputs_1, x, name="RNN_model")
rnn_model.summary()





In [ ]:

# VAE model

vae_input = tf.keras.layers.Input(shape=(5,), name="VAE_input")
vae_decoder_output = rnn_model(model(vae_input))


In [ ]:

def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = tf.reduce_mean(x, axis=1, keepdims=True)
    my = tf.reduce_mean(y, axis=1, keepdims=True)
    xm, ym = x - mx, y - my
    t1_norm = tf.nn.l2_normalize(xm, axis = 1)
    t2_norm = tf.nn.l2_normalize(ym, axis = 1)
    cosine = tf.losses.cosine_distance(t1_norm, t2_norm, axis = 1)
    return cosine


def get_mse(y_true, y_pred):
    return tf.keras.losses.MSE(y_true, y_pred)


def get_correlation(y_true, y_pred):
    return tf.contrib.metrics.streaming_pearson_correlation(y_pred, y_true)



In [ ]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
                         loss=tf.keras.losses.log_cosh,
                         metrics=[tf.keras.metrics.CosineSimilarity(axis=-1)]) 

model.fit(train_set,
          validation_data=val_set,
          epochs=200)

model.evaluate(test_set)
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/signal_prediction/h5_models/')
model.save('ann_model_signal_prediction_VAE_2.h5')
